In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==1.15.2

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, auc

In [ ]:
import seaborn as sns

In [ ]:
path = "../input/emploeey-datasets/0 (1).csv"
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df)

In [ ]:
# target value
df["income_bracket"].unique()

In [ ]:
def fix_label(label):
  if label == ' <=50K':
    return 0
  else:
    return 1

In [ ]:
df["income_bracket"] = df["income_bracket"].apply(fix_label)

In [ ]:
df["income_bracket"].unique()

In [ ]:
X = df.drop('income_bracket', axis=1)
y = df['income_bracket']

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [ ]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender",["Feamale", "Male"])
race = tf.feature_column.categorical_column_with_vocabulary_list("race",[' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',' Other'])

workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [ ]:
feature_columns = [age, workclass, education, education_num, marital_status, occupation,
             relationship, race, gender, capital_gain, capital_loss, hours_per_week, native_country]

In [ ]:
train_input = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)
test_input = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=True)

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
model.train(input_fn=train_input, steps=100)

In [ ]:
predictions = list(model.predict(input_fn=test_input))

In [ ]:
final_predict = []
for pred in predictions:
  final_predict.append(pred['class_ids'])

In [ ]:
print(classification_report(y_test, final_predict))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, final_predict)
roc_auc = auc(fpr, tpr)

In [ ]:
print(f"ROC AUC Score: { roc_auc }")